In [1]:
# configure spark variables
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
    
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

# load up other dependencies
import re
import pandas as pd

import glob
import matplotlib.pyplot as plt
import numpy as np

22/11/27 22:49:31 WARN Utils: Your hostname, tariro-ThinkPad-T440p resolves to a loopback address: 127.0.1.1; using 192.168.100.11 instead (on interface wlp4s0)
22/11/27 22:49:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/27 22:49:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/tariro/.local/lib/python3.10/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
import os
if not os.path.exists("images"):
    os.mkdir("images")

import os
import glob
import plotly.express as px
from plotly.subplots import make_subplots
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [6]:
html_output_file = '../index.html'
with open(html_output_file, 'w') as f:
    f.write('<h1>Open GiLAN Testbed Results</h1>')
    f.write('<h3> The graphs summaries the NFV performance metrics<h3>')
    f.write('<h4> CPU  <h4>')
    f.write('<h5><a href="#cpu-metrics"> Skip to results </a></h5>')
    f.write('<p> The CPU is responsible for executing all workloads on the NFV. Like other resources, the CPU is managed by the kernel. The user-level applications access CPU resources by sending system calls to the kernel. The kernel also receives other system call requests from different processes; memory loads and stores can issue page faults system calls. The primary consumers of CPU resources are threads (also called tasks), which belong to procedures, kernel routines and interrupt routes. The kernel manages the sharing via a CPU scheduler.</p>')
    f.write('<p> There are three thread states: ON-PROC for threads running on a CPU, RUNNABLE for threads that could run but are waiting their turn, and SLEEP for blocked lines on another event, including uninterruptible waits. These can be categorised into two for more accessible analysis, on-CPU referring to ON-PROC, and off-CPU referring to all other states, where the thread is not running on a CPU. Lines leave the CPU in one of two ways: (1) voluntary if they block on I/O, a lock, or asleep, or (2) involuntary if they have exceeded their scheduled allocation of CPU time. When a CPU switches from running one process or thread to another, it switches address spaces and other metadata. This process is called context switching; it also consumes the CPU resources. All these processes, described, in general, consume the CPU time. In addition to the time, another CPU resource used by the methods, kernel routines and interrupts routines is the CPU cache.</p>')
    f.write('<p> There are typically multiple levels of CPU cache, increasing in both size and latency. The caches end with the last-level store (LLC), large (Mbytes) and slower. On a processor with three levels of supplies, the LLC is also the Level 3 cache. Processes are instructions to be interpreted and run by the CPU. This set of instructions is typically loaded from RAM and cached into the CPU cache for faster access. The CPU first checks the lower cache, i.e., L1 cache. If the CPU finds the data, this is called a hit. If the CPU does not see the data, it looks for it in L2 and then L3. If the CPU does not find the data in any memory caches, it can access it from your system memory (RAM). When that happens, it is known as a cache miss. In general, a cache miss means high latency, i.e., the time needed to access data from memory. </p>')

    f.write('<h4> Memory <h4>')
    f.write('<h5><a href="#memory-metrics"> Skip to results </a> </h5>')
    f.write('<p> The kernel and processor are responsible for mapping the virtual memory to physical memory. For efficiency, memory mappings are created in groups of memory called <em>pages</em>. When an application starts, it begins with a request for memory allocation. In the case that there is no free memory on the heap, the syscall <em>brk()</em> is issued to extend the size of the bank. However, if there is free memory on the heap, a new memory segment is created via the <em>mmap()</em> syscall. Initially, this virtual memory mapping does not have a corresponding physical memory allocation. Therefore when the application tries to access this allocated memory segment, the error called <em>page fault</em> occurs on the MMU. The kernel then handles the page fault, mapping from the virtual to physical memory. The amount of physical memory allocated to a process is called resident set size (RSS). When there is too much memory demand on the system, the kernel page-out daemon (kswapd) may look for memory pages to free. Three types of pages can be released in their order: pages that we read but not modified (backed by disk) these can be immediately rid, pages that have been modified (dirty) these need to be written to disk before they can be freed and pages of application memory (anonymous) these must be stored on a swap device before they can be released. kswapd, a page-out daemon, runs periodically to scan for inactive and active pages with no memory to free. It is woken up when free memory crosses a low threshold and goes back to sleep when it crosses a high threshold. Swapping usually causes applications to run much more slowly.</p>')

    f.write('<h4>Filesytem <h4>')
    f.write('<h5><a href="#filesystem-metrics"> Skip to results </a> </h5>')
    f.write('<p> The file system that applications usually interact with directly and file systems can use caching, read-ahead, buffering, and asynchronous I/O to avoid exposing disk I/O latency to the application. Logical I/O describes requests to the file system. If these requests must be served from the storage devices, they become physical I/O. Not all I/O will; many logical read requests may be returned from the file system cache and never become physical I/O. File systems are accessed via a virtual file system (VFS). It provides operations for reading, writing, opening, closing, etc., which are mapped by file systems to their internal functions. Linux uses multiple caches to improve the performance of storage I/O via the file system. These are Page cache: This contains virtual memory pages and enhances the performance of file and directory I/O. Inode cache, which are data structures used by file systems to describe their stored objects. The directory cache caches mappings from directory entry names to VFS inodes, improving the performance of pathname lookups. The page cache grows to be the largest of all these because it caches the contents of files and includes “dirty” pages that have been modified but not yet written to disk.</p>')

    f.write('<h4>Disk I/O <h4>')
    f.write('<h5><a href="#disk-metrics"> Skip to results </a> </h5>')
    f.write('<p> Linux exposes rotational magnetic media, flash-based storage, and network storage as storage devices. Therefore, disk I/O refers to I/O operations on these devices. Disk I/O is a common source of performance issues because I/O latency on storage devices is orders of magnitude slower than the nanosecond or microsecond speed of CPU and memory operations. Block I/O refers to device access in blocks. I/O is queued and scheduled in the block layer. The wait time is spent in the block layer scheduler queues and device dispatcher queues from the operating system. Service time is the time from device issue to completion. This may include the time spent waiting in an on-device line. Request time is the overall time from when an I/O was inserted into the OS queues to its completion. The request time matters the most, as that is the time that applications must wait if I/O is synchronous.</p>')

    f.write('<h4>Networking<h4>')
    f.write('<h5><a href="#networking-metrics"> Skip to results </a> </h5>')
    f.write('<p> Networking is a complex part of the Linux system. It involves many different layers and protocols, including the application, protocol libraries, syscalls, TCP or UDP, IP, and device drivers for the network interface. In general, the Networking system can be broken down into four. The NIC and Device Driver Processing first reads packets from the NIC and puts them into kernel buffers. Besides the NIC and Device driver, this process includes the DMA and particular memory regions on the RAM for storing receive and transmit packets called rings and the NAPI system for poling packets from these rings to the kernel buffers. It also incorporates some early packet processing hooks like XDP and AF\_XDP and can have custom drivers that bypass the kernel (i.e., the following two processes) like DPDK. Following is the Socket processing. This part also includes queuing and different queuing disciplines. It also incorporates some packet processing hooks like TC, Netfilter etc., which can alter the flow of the networking stack. After that is the  Protocol processing layer, which applies functions for different IP and transport protocols, both these protocols run under the context of SoftIrq. Lastly is the application process. The application receives and sends packets on the destination socket</p>')
    
    f.write('<h4>Flame Graphs to analyse code paths<h4>')
    f.write('<h5><a href="#flame-graphs"> Skip to results </a> </h5>')
    f.write('<p> A flame graph visualizes a distributed request trace and represents each service call that occurred during the requests execution path with a timed, color-coded, horizontal bar. Flame graphs for distributed traces include error and latency data to help developers identify and fix bottlenecks in their applications..</p>')

In [4]:
with open(html_output_file, 'a') as f:
    f.write('<h2 id="cpu-metrics"> CPU Metrics </h2>')
    f.write('<p> To analyse the performance of the CPU along with the performance of the processes, kernel routines or interrupt routines, it is essential to look at some key insights from the performance data. Firstly, it is necessary to know <em>the processes that have run and their age/lifespan</em>. This can also be used to identify issues with short-lived processes. After which, it is essential to look at the <em>time each of the methods spends waiting for its turn on the CPU</em>. Given this information, we can also find the time that the processes spend off-CPU either voluntary or involuntary leaving the CPU, lastly in cases where we see performance issues from this data, e.g., a process consuming a lot of time on-CPU or a process spending a lot of time off-CPU. It would be helpful to analyse all <em>code paths that are consuming CPU resources</em> and the <em>code path resulting in the process being off-CPU</em>.</p>')
    f.write('<p> As mentioned earlier, another CPU resource outside time is the CPU cache. This can also be a factor for processes with multiple lifespans. To diagnose this, it is essential to get the <em>number of cache misses of the LLC</em> as well as <em>the hit ratios of the LLC</em>. However this two metrics cannot be collected on virtual machines like in the case of Cloud-Based NFVI.  The prototyping system in this study collects, and presents this data, both as summary graphs and raw data from analysis </p>')

In [25]:
runqlat_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=runqlat").select(
    "kpps", "ts", "val_type", "tool",
    explode("data").alias("dataExplode")
).orderBy(col('kpps')).select(col("kpps").cast(StringType()), "ts", "val_type", "tool", "dataExplode.*", concat_ws('-',col('dataExplode.interval-start'),col('dataExplode.interval-end')).alias("interval"))
runqlat_fig = px.bar(runqlat_df.toPandas(), x="interval", y="count", color="kpps", barmode="group",
        title="Time each of the processes spends waiting for its turn on CPU",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of Processes",
                    "interval": "Latency (usecs)"
        })
runqlat_fig.show()
runqlat_fig.write_image("images/runqlat.jpeg")
with open(html_output_file, 'a') as f:
    f.write(runqlat_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [26]:
with open(html_output_file, 'a') as f:
    f.write('<div><a href="#profile">See code paths that are consuming CPU resources</a></div>')

In [27]:
with open(html_output_file, 'a') as f:
    f.write('<div><a href="#offcputime">See code paths casing processes to go Off-CPU</a></div>')

In [28]:
with open(html_output_file, 'a') as f:
    f.write('<h2 id="memory-metrics"> Memory Metrics </h2>')
    f.write('<p> Memory operations can be frequent; therefore, to reduce overheads, it is important to look at some of the non-frequent events that can give insights into the performance of the memory resource. The relatively infrequent activities are: <em>brk()</em> and <em>mmap()</em> calls, <em>page faults</em>, and <em>page-outs</em>. An important is to know the number of memory requests that result in a new segment on the heap, i.e., <em>requests for mappings</em>. Following, it is beneficial to know the <em>code path responsible for heap extension</em> which can review the portion that resulted in extending the heap. Another import operation is page fault, which results in latency and growth of a process RSS. Likewise, it is important to know the <em>code path responsible for page faults</em>. As the system reclaims memory, we would also want to know the <em>process affected and the latency: the time taken for the reclaim</em>. </p>')

In [29]:
data_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=sys_enter_brk")

schema = StructType([
  StructField('type', StringType(), True),
  StructField('kpps', StringType(), True),
  StructField('count', IntegerType(), True),
  StructField('process', IntegerType(), True),
  ])

#Create empty DataFrame directly.
sys_enter_brk_df = spark.createDataFrame([], schema)

columns = data_df.select("data.@.*").columns
data_df = data_df.select("*", "data.@.*").drop("data")

for c in columns:
  try:
    df = data_df.select("type", "kpps", col(c).alias('count')).withColumn('process', lit(c))
    sys_enter_brk_df = sys_enter_brk_df.union(df)
  except:
    print("Failed to find {</em>".format(c))

sys_enter_brk_fig = px.sunburst(sys_enter_brk_df.orderBy(col('kpps')).toPandas(), path=['kpps', 'process'], values='count',
                  color='process', hover_data=['count'],
                  title="Processes responsible for heap extension")
sys_enter_brk_fig.show()
sys_enter_brk_fig.write_image("images/sys_enter_brk.jpeg")
with open(html_output_file, 'a') as f:
    f.write(sys_enter_brk_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [30]:
with open(html_output_file, 'a') as f:
    f.write('<div><a href="#page_fault_user">See user code path responsible for page faults</a></div>')
    f.write('<div><a href="#page_fault-kernel">See kernel code path responsible for page faults</a></div>')

In [31]:
with open(html_output_file, 'a') as f:
    f.write('<h2 id="filesystem-metrics"> Filesystem Metrics </h2>')
    f.write('<p> Firstly, we would want to <em>characterisation virtual file system operations</em>. This helps us know what the process is spending the most time doing on the filesystem - reads and writes (I/O), creates, opens, and syncs. After that, it is essential to know the size of data in for the <em>read and write operations by the process names</em>. This can assist in diagnosing the process responsible for the degraded filesystem performance in such cases. In the same manner, in the case of frequent VFS open operation, it is essential to know the <em>processes responsible for opening files</em>. While the earlier results can help in understanding the processes, it is also necessary to know the <em>filenames with the most frequently read and written</em>. This at a high level can expose some configuration errors, for example, verbose logging in production. This is the case for the Bind9 VNF in our case. Since sockets are also perceived as filenames, this can also show the frequency of sockets reads and writes. As mentioned earlier, the filesystem uses cache to avoid exposing disk I/O latency; therefore, another critical performance factor to consider is how the stock is performing. Applications are affected mainly by page cache; examining <em>the page cache hit ratio over time</em> can give insights on the NFV configuration tuning needed.</p>')

In [32]:
vfsstat_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=vfsstat")
vfsstat_fig = px.bar(vfsstat_df.toPandas(), x="kpps", y=["WRITE", "READ", "OPEN", "FSYNC", "CREATE"], barmode='group',
            title="High-level workload characterization of virtual file system operations",
            labels={
                     "kpps": "Packets Per Second (thousand)",
                     "value": "Count",
                     "variable": "VFS operation"
            })
vfsstat_fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
vfsstat_fig.show()
vfsstat_fig.write_image("images/vfsstat.jpeg")
with open(html_output_file, 'a') as f:
    f.write(vfsstat_fig.to_html(full_html=False, include_plotlyjs='cdn'))


In [13]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=vfssize")
schema = StructType([
  StructField('type', StringType(), True),
  StructField('kpps', IntegerType(), True),
  StructField('count', IntegerType(), True),
  StructField('min', IntegerType(), True),
  StructField('max', IntegerType(), True),
  StructField('device', StringType(), True),
  StructField('program', StringType(), True),
  StructField('filename', StringType(), True),
  ])

#Create empty DataFrame directly.
vfssize_df = spark.createDataFrame([], schema)

columns = data_df.select("data.@.*").columns
columns = [c.replace('.', '_') for c in columns]
data_df = data_df.select("*", "data.@.*").drop("data")
data_df = data_df.toDF(*(c.replace('.', '_') for c in data_df.columns))
for c in columns:
  try:
    df = data_df.select("*", explode(c).alias("dataExplode")).select("type", "kpps", "dataExplode.*").withColumn('device', lit(c)).withColumn('program', lit(c.split(',')[0])).withColumn('filename', lit(c.split(',')[1]))
    vfssize_df = vfssize_df.union(df)
  except:
    print("Failed to find {}".format(c))

vfssize_df = vfssize_df.orderBy(col('kpps')).select("*", concat_ws(' - ',col('min'),col('max')).alias("interval"))

vfssize_fig_1 = px.scatter(vfssize_df.toPandas(), x="interval", y="kpps",
	         size="count", color="program",
             title="The frequency of I/O per proccess name per I/O size",
            labels={
                        "kpps": "Packets Per Second (thousand)",
                        "count": "Number of I/O operations",
                        "interval": "I/O Size (bytes)"
                    })
vfssize_fig_1.show()
vfssize_fig_1.write_image("images/vfssize-process.jpeg")
vfssize_fig_2 = px.scatter(vfssize_df.toPandas(), x="interval", y="kpps",
	         size="count", color="filename",
             title="The frequency of I/O per filename per I/O size",
            labels={
                        "kpps": "Packets Per Second (thousand)",
                        "count": "Number of I/O operations",
                        "interval": "I/O Size (bytes)"
                    })
vfssize_fig_2.show()
vfssize_fig_2.write_image("images/vfssize-filename.jpeg")
pd_df = vfssize_df.toPandas()
pd_df['kpps'] = pd_df['kpps'].astype(str)
vfssize_fig_3 = px.bar(pd_df, x="interval", y="count", color="kpps", barmode="group",
        title="Time each of the processes spends waiting for its turn on CPU",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of I/O operations",
                    "interval": "I/O Size (bytes)"
                })
vfssize_fig_3.show()
vfssize_fig_3.write_image("images/vfssize.jpeg")
with open(html_output_file, 'a') as f:
    f.write(vfssize_fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(vfssize_fig_2.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(vfssize_fig_3.to_html(full_html=False, include_plotlyjs='cdn'))

22/11/27 21:57:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Exception in thread "refresh progress" java.lang.OutOfMemoryError: Java heap space
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.status.AppStatusStore.activeStages(AppStatusStore.scala:114)
	at org.apache.spark.ui.ConsoleProgressBar.org$apache$spark$ui$ConsoleProgressBar$$refresh(ConsoleProgressBar.scala:64)
	at org.apache.spark.ui.ConsoleProgressBar$$anon$1.run(ConsoleProgressBar.scala:52)
	at java.base/java.util.TimerThread.mainLoop(Timer.java:556)
	at java.base/java.util.TimerThread.run(Timer.java:506)


Failed to find networkctl,libudev_so_1_6_17
Failed to find networkctl,libunistring_so_2_1_0


Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: Java heap space
Exception in thread "netty-rpc-env-timeout" java.lang.OutOfMemoryError: Java heap space
22/11/27 22:29:49 ERROR Utils: Uncaught exception in thread driver-heartbeater
java.lang.OutOfMemoryError: Java heap space


Failed to find networkd-dispat,UNIX


22/11/27 22:30:16 ERROR Utils: uncaught error in thread Spark Context Cleaner, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
22/11/27 22:30:16 ERROR Utils: throw uncaught fatal error in thread Spark Context Cleaner
java.lang.OutOfMemoryError: Java heap space
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space


Failed to find networkd-dispat,[eventfd]
Failed to find nrf-healthcheck,FIFO
Failed to find nrf-healthcheck,netstat
Failed to find nrf-healthcheck,nrf-healthcheck_sh
Failed to find oai_amf,FIFO
Failed to find oai_ausf,FIFO
Failed to find runc,_0e517e82567d774bdd46616236c4daafe7b8d79e827b722e0097d51ceca52d
Failed to find runc,_171ad9ceb2167a1956c84de316c8e7c2a2595a881749f3bd62b56ad18b4821
Failed to find runc,_1894ac320519ae46484b93834a5177f7cd02bfe3315bd6ca8076b1afedf238
Failed to find runc,_1a2a6d88b9af09f5153604967689f10eff6277c211a12e40f4df19e8ec14ba
Failed to find runc,_2184e2389761704e6f98a6a07cdd7440334d113fefca627593c6daf6a33b5f
Failed to find runc,_4190e8c438522bd9265b474b8e7cb88c1705ccbdf476e837dff1b5ec74ee3f
Failed to find runc,_4641b220cd56f0f594385b4e296c5b778aaaeb90817d0a5f9f088513100520
Failed to find runc,_466490d3a95eb99029ef2a4299af502afacab73766187fc5f2653e5ab077ab
Failed to find runc,_466bb974347493c328cadc1d7eb5d196aba8affd3acd3e63a604ea57e9d8c6
Failed to find runc,_

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/tariro/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/tariro/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Failed to find runc,_4ac6e65ae968b5056160f9c53cd37e1b754fe05af3b8f8cb58abf7826177ef


In [7]:
filetop_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=filetop")
# Reads
filetop_read_fig = px.sunburst(filetop_df.toPandas(), path=['kpps', 'FILE'], values='R_Kb',
                  color='FILE', hover_data=['READS'],
                  title="Most frequently read filenames - including TCP sockets",
                  labels={
                    "kpps": "Packets Per Second (thousand)",
                    "READS": "Number of file reads",
                    "FILE": "File names"
                })
filetop_read_fig.show()
filetop_read_fig.write_image("images/filetop-reads.jpeg")
# Writes
filetop_write_fig = px.sunburst(filetop_df.toPandas(), path=['kpps', 'FILE'], values='W_Kb',
                  color='FILE', hover_data=['WRITES'],
                  title="Most frequently written filenames - including TCP sockets")
filetop_write_fig.show()
filetop_write_fig.write_image("images/filetop-writes.jpeg")
with open(html_output_file, 'a') as f:
    f.write(filetop_read_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(filetop_write_fig.to_html(full_html=False, include_plotlyjs='cdn'))

ValueError: ('Non-leaves rows are not permitted in the dataframe \n', kpps    30
FILE      
Name: 1600, dtype: object, 'is not a leaf.')

In [8]:
cachestat_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=cachestat")
cachestat_fig = px.bar(cachestat_df.toPandas(), x="kpps", y=["HITS", "MISSES", "DIRTIES"], barmode='group',
            title="Page cache hit vs miss ratio at different packet rate",
            labels={
                     "kpps": "Packets Per Second (thousand)",
                     "value": "Count",
                     "variable": "Cache Operation"
                 })
cachestat_fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
cachestat_fig.show()
cachestat_fig.write_image("images/cachestat.jpeg")
with open(html_output_file, 'a') as f:
    f.write(cachestat_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [9]:
with open(html_output_file, 'a') as f:
    f.write('<h2 id="disk-metrics"> Disk I/O Metrics </h2>')
    f.write('<p> To first evaluate the performance of Disk I/O, we would want to know the overall <em>block I/O device latency</em>. This refers to the time from issuing a request to the device, to when it completes, including time spent queued in the operating system. While latency can show the overall performance of the Disk I/O, to remediate them, it would be essential to get more details, like the <em>processes performing I/O on disk and the bite-size</em>. In addition to the overall latency request time, it is also necessary to know the <em>time requests were queued in the I/O scheduler in the block layer</em>.</p>')

In [10]:
biolatency_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=biolatency").select(
    "kpps", "ts", "val_type", "tool",
    explode("data").alias("dataExplode")
).orderBy(col('kpps')).select(col("kpps").cast(StringType()), "ts", "val_type", "tool", "dataExplode.*", concat_ws('-',col('dataExplode.interval-start'),col('dataExplode.interval-end')).alias("interval"))
biolatency_fig = px.bar(biolatency_df.toPandas(), x="interval", y="count", color="kpps", barmode="group",
        title="Block I/O device latency",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of I/O",
                    "interval": "Latency (usecs)"
                })
biolatency_fig.show()
biolatency_fig.write_image("images/biolatency.jpeg")
with open(html_output_file, 'a') as f:
    f.write(biolatency_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [11]:
with open(html_output_file, 'a') as f:
    f.write('<h2 id="networking-metrics"> Netowrking Metrics </h2>')
    f.write('<p> To draw insights on the performance of the networking on the NFV and for the VNF, the initial bit is that we will need to know the <em>number of packets being received and their sizes</em>. After that, we would want to see the <em>latency of the device queue</em>, i.e., the time from when the packets have been pushed into the device layer for sending until the packages are sent out as signalled by NAPI. After that, we would want to know the <em>time spent on the queuing disciplines</em>. Next, we would want to see the <em>latency for IP protocol connections and the process of making the connection</em>. Following this, we would also like to know the <em>lifespan of the kernel buffers</em> that are used to pass packets across the networking stack. This can show the latency within the networking stack. While this shows the latency, it won’t show the packet drops. For this, it would be essential to know <em>the number of packets and allocated size of the socket buffers and their limits</em>. Packets are dropped when the socket limits have been reached. </p>')

In [12]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=netsize")
schema = StructType([
  StructField('type', StringType(), True),
  StructField('kpps', IntegerType(), True),
  StructField('count', IntegerType(), True),
  StructField('min', IntegerType(), True),
  StructField('max', IntegerType(), True),
  StructField('device', StringType(), True),
  StructField('action', StringType(), True),
  ])

#Create empty DataFrame directly.
netsize_df = spark.createDataFrame([], schema)

types = data_df.select("data.*").columns
data_df = data_df.select("*", "data.*").drop("data")
for t in types: # for action of the actions -> e.g., recv_bytes
  columns = data_df.select("{}.*".format(t)).columns
  for c in columns: # for each device -> e.g., eth0
      df = data_df.select("type", "kpps", "tool", explode("{}.{}".format(t, c)).alias("dataExplode")).select("type", "kpps", "dataExplode.*").withColumn('device', lit(c)).withColumn('action', lit(t))
      netsize_df = netsize_df.union(df)

netsize_df = netsize_df.orderBy(col('kpps')).select("*", concat_ws(' - ',col('min'),col('max')).alias("interval"))
pd_df = netsize_df.toPandas()
pd_df['kpps'] = pd_df['kpps'].astype(str)
netsize_fig = px.bar(pd_df, x="interval", y="count", color="kpps", barmode="group", facet_col="action",
        title="Packet Distribution",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of Packets",
                    "interval": "Packet Size (bytes)"
                })
netsize_fig.show()
netsize_fig.write_image("images/netsize.jpeg")
with open(html_output_file, 'a') as f:
    f.write(netsize_fig.to_html(full_html=False, include_plotlyjs='cdn'))

22/11/27 22:52:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/11/27 22:53:05 WARN DAGScheduler: Broadcasting large task binary with size 1783.9 KiB
22/11/27 22:53:35 WARN DAGScheduler: Broadcasting large task binary with size 1786.0 KiB
22/11/27 22:53:58 WARN DAGScheduler: Broadcasting large task binary with size 1547.5 KiB


In [13]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=nettxlat-dev")
schema = StructType([
  StructField('type', StringType(), True),
  StructField('kpps', IntegerType(), True),
  StructField('count', IntegerType(), True),
  StructField('min', IntegerType(), True),
  StructField('max', IntegerType(), True),
  StructField('device', StringType(), True),
  ])

#Create empty DataFrame directly.
nettxlat_df = spark.createDataFrame([], schema)

columns = data_df.select("data.@us.*").columns
data_df = data_df.select("*", "data.@us.*").drop("data")

for c in columns: # for each device -> e.g., eth0
    df = data_df.select("type", "kpps", "tool", explode(c).alias("dataExplode")).select("type", "kpps", "dataExplode.*").withColumn('device', lit(c))
    nettxlat_df = nettxlat_df.union(df)

nettxlat_df = nettxlat_df.orderBy(col('kpps')).select("*", concat_ws(' - ',col('min'),col('max')).alias("interval"))
pd_df = nettxlat_df.toPandas()
pd_df['kpps'] = pd_df['kpps'].astype(str)
nettxlat_fig = px.bar(pd_df, x="interval", y="count", color="kpps", barmode="group", facet_row="device",
        title="Latency of the device queue",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of Packets",
                    "interval": "Latency (usecs)"
                })
nettxlat_fig.show()
nettxlat_fig.write_image("images/nettxlat.jpeg")
with open(html_output_file, 'a') as f:
    f.write(nettxlat_fig.to_html(full_html=False, include_plotlyjs='cdn'))

AnalysisException: Found duplicate column(s) in the data schema: `eth0`

In [14]:
data_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=skblife")
schema = StructType([
  StructField('kpps', IntegerType(), True),
  StructField('type', IntegerType(), True),
  StructField('count', IntegerType(), True),
  StructField('min', IntegerType(), True),
  StructField('max', IntegerType(), True),
  StructField('interval', StringType(), True),
  ])

#Create empty DataFrame directly.
skblife_df = spark.createDataFrame([], schema)

data_df = data_df.select("*", explode('data.@skb_residency_nsecs').alias("dataExplode")).select("type", "kpps", "dataExplode.*")
data_df = data_df.orderBy(col('kpps')).select(col("kpps").cast(StringType()), "type", "count", "min", "max", concat_ws(' - ',col('min'),col('max')).alias("interval"))
skblife_df = skblife_df.union(data_df)

skblife_fig = px.bar(skblife_df.toPandas(), x="interval", y="count", color="kpps", barmode="group",
        title="Distribution of lifespan or age of the kernel buffers",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of kernel buffers",
                    "interval": "Lifetime/Age (usecs)"
                })
skblife_fig.show()
skblife_fig.write_image("images/skblife.jpeg")
with open(html_output_file, 'a') as f:
    f.write(skblife_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [15]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data_df = spark.read.option("basePath", "../ansible/.results").json("../ansible/.results/kpps=*/tool=sormem")
schema = StructType([
  StructField('type', StringType(), True),
  StructField('kpps', IntegerType(), True),
  StructField('count', IntegerType(), True),
  StructField('min', IntegerType(), True),
  StructField('max', IntegerType(), True),
  StructField('action', StringType(), True),
  ])

#Create empty DataFrame directly.
sormem_df = spark.createDataFrame([], schema)

columns = data_df.select("data.*").columns
data_df = data_df.select("*", "data.*").drop("data")
for c in columns:
    df = data_df.select("*", explode(c).alias("dataExplode")).select("type", "kpps", "dataExplode.*").withColumn('action', lit(c))
    sormem_df = sormem_df.union(df)

sormem_df = sormem_df.orderBy(col('kpps')).select("*", concat_ws(' - ',col('min'),col('max')).alias("interval"))
pd_df = sormem_df.toPandas()
pd_df['kpps'] = pd_df['kpps'].astype(str)
sormem_fig = px.bar(pd_df, x="interval", y="count", color="kpps", barmode="group", facet_col="action",
        title="The number of packets and allocated size of the socket buffers and their limits",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of Sock RCV calls",
                    "interval": "Buffer Size"
                })
sormem_fig.show()
sormem_fig.write_image("images/sormem.jpeg")
with open(html_output_file, 'a') as f:
    f.write(sormem_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [16]:
hardirqs_count_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=hardirqs_count")
# print(df.rdd.getNumPartitions())
import plotly.express as px
hardirqs_count_fig = px.bar(hardirqs_count_df.toPandas(), x="kpps", y="count", color="hardirq",
            title="Total time spent servicing hard IRQs (hard interrupts)",
            labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Time (usecs)",
                    "hardirq": "Hardware Interupts"
                })
hardirqs_count_fig.show()
hardirqs_count_fig.write_image("images/hardirqs_count.jpeg")
with open(html_output_file, 'a') as f:
    f.write(hardirqs_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [17]:
hardirqs_dist_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=hardirqs_dist").select(
    "kpps", "ts", "val_type", "tool",
    explode("data").alias("dataExplode")
).orderBy(col('kpps')).select(col("kpps").cast(StringType()), "ts", "val_type", "tool", "dataExplode.*", concat_ws('-',col('dataExplode.interval-start'),col('dataExplode.interval-end')).alias("interval"))
import plotly.express as px
hardirqs_dist_fig = px.bar(hardirqs_dist_df.toPandas(), x="interval", y="count", color="kpps", barmode="group",
        title="The number of Hardware IRQ per duration distribution",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of Hardware IRQs",
                    "interval": "Time (usecs)"
                })
hardirqs_dist_fig.show()
hardirqs_dist_fig.write_image("images/hardirqs_dist.jpeg")
with open(html_output_file, 'a') as f:
    f.write(hardirqs_dist_fig.to_html(full_html=False, include_plotlyjs='cdn'))

AnalysisException: cannot resolve 'ts' given input columns: [_corrupt_record, kpps, tool];
'Project [kpps#6355, 'ts, 'val_type, tool#6356, explode('data) AS dataExplode#6360]
+- Relation [_corrupt_record#6354,kpps#6355,tool#6356] json


In [18]:
softirqs_count_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=softirqs_count")
# print(df.rdd.getNumPartitions())
import plotly.express as px
softirqs_count_fig = px.bar(softirqs_count_df.toPandas(), x="kpps", y="count", color="softirq",
            title="Total time spent servicing soft IRQs",
            labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Time (usecs)",
                    "hardirq": "Hardware Interupts"
                })
softirqs_count_fig.show()
softirqs_count_fig.write_image("images/softirqs_count.jpeg")
with open(html_output_file, 'a') as f:
    f.write(softirqs_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [19]:
softirqs_dist_df = spark.read.option("basePath", "../ansible/.results").json(
"../ansible/.results/kpps=*/tool=softirqs_dist").select(
    "kpps", "ts", "val_type", "tool",
    explode("data").alias("dataExplode")
).orderBy(col('kpps')).select(col("kpps").cast(StringType()), "ts", "val_type", "tool", "dataExplode.*", concat_ws('-',col('dataExplode.interval-start'),col('dataExplode.interval-end')).alias("interval"))
import plotly.express as px
# softirqs_dist_df = softirqs_dist_df.select("*", concat_ws(' - ',col('interval-start'),col('interval-end')).alias("interval"))
softirqs_dist_fig = px.bar(softirqs_dist_df.toPandas(), x="interval", y="count", color="kpps", barmode="group",
        title="The number of SOFTIRQ per duration distribution",
        color_discrete_sequence=px.colors.sequential.Plasma,
        labels={
                    "kpps": "Packets Per Second (thousand)",
                    "count": "Number of Soft IRQs",
                    "interval": "Time (usecs)"
                })
softirqs_dist_fig.show()
softirqs_dist_fig.write_image("images/softirqs_dist.jpeg")
with open(html_output_file, 'a') as f:
    f.write(softirqs_dist_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [34]:
import base64

file_list = glob.glob('../ansible/.results/kpps=*/tool=profile/*.txt')
with open(html_output_file, 'a') as f:
    f.write('<div id="profile">')
    for file in file_list:
        try:
            kpps = re.search('../ansible/.results/kpps=(.+?)/tool=profile/(.+?).txt', file).group(1)
            os.system("../FlameGraph/flamegraph.pl --color=java --title='On-CPU Time Flame Graph for {} kpps' < {} > images/{}kpps_profile.svg".format(kpps, file, kpps))
            with open("images/{}kpps_profile.svg".format(kpps), "rb") as image_file:
                kpps_profile = str(base64.b64encode(image_file.read()),'utf-8')
                f.write('<img id="1kpps_profile" src="data:image/svg+xml;base64,{}" />'.format(kpps_profile))
        except AttributeError:
            print("Error creating profile svg files")

    f.write('</div>')



In [21]:
with open(html_output_file, 'a') as f:
    f.write('<div id="offcputime">')
    file_list = glob.glob('../ansible/.results/kpps=*/tool=offcputime/*.txt')
    for file in file_list:
        try:
            kpps = re.search('../ansible/.results/kpps=(.+?)/tool=offcputime/(.+?).txt', file).group(1)
            os.system("../FlameGraph/flamegraph.pl --hash --bgcolors=blue --title='Off-CPU Time Flame Graph for {} kpps' < {} > images/{}kpps_offcputime.svg".format(kpps, file, kpps))
            with open("images/{}kpps_offcputime.svg".format(kpps), "rb") as image_file:
                kpps_offcputime = str(base64.b64encode(image_file.read()),'utf-8')
                f.write('<img src="data:image/svg+xml;base64,{}" />'.format(kpps_offcputime))
        except AttributeError:
            print("Error creating offcputime svg files")
    f.write('</div>')

In [22]:

file_list = glob.glob('../ansible/.results/kpps=*/tool=page_fault_user/*.txt')
with open(html_output_file, 'a') as f:
    f.write('<div id="page_fault_user">')
    for file in file_list:
        try:
            kpps = re.search('../ansible/.results/kpps=(.+?)/tool=page_fault_user/(.+?).txt', file).group(1)
            os.system("../FlameGraph/flamegraph.pl --hash --width=800 --title='Page Fault Flame Graph for {} kpps' --colors=java --bgcolor=green < {} > images/{}kpps_page_fault_user.svg".format(kpps, file, kpps))
            with open("images/{}kpps_page_fault_user.svg".format(kpps), "rb") as image_file:
                kpps_page_fault_user = str(base64.b64encode(image_file.read()),'utf-8')
                f.write('<img src="data:image/svg+xml;base64,{}" />'.format(kpps_page_fault_user))
        except AttributeError:
            print("Error creating page_fault_user svg files")
    f.write('</div>')

In [23]:
file_list = glob.glob('../ansible/.results/kpps=*/tool=page_fault_kernel/*.txt')
with open(html_output_file, 'a') as f:
    f.write('<div id="page_fault_kernel">')
    for file in file_list:
        try:
            kpps = re.search('../ansible/.results/kpps=(.+?)/tool=page_fault_kernel/(.+?).txt', file).group(1)
            os.system("../FlameGraph/flamegraph.pl --hash --width=800 --title='Kernel Page Fault Flame Graph for {} kpps' --colors=java --bgcolor=green < {} > images/{}kpps_page_fault_kernel.svg".format(kpps, file, kpps))
            with open("images/{}kpps_page_fault_kernel.svg".format(kpps), "rb") as image_file:
                kpps_page_fault_kernel = str(base64.b64encode(image_file.read()),'utf-8')
                f.write('<img src="data:image/svg+xml;base64,{}" />'.format(kpps_page_fault_kernel))
        except AttributeError:
            print("Error creating page_fault_kernel svg files")
    f.write('</div>')

In [41]:
import base64

# with open('images/1kpps_profile.svg', "rb") as image_file:
#     kpps_profile = str(base64.b64encode(image_file.read()),'utf-8')

with open('p_graph.html', 'a') as f:
    f.write('<h1>Open GiLAN Testbed Results</h1>')
    f.write('<div><a href="#1kpps_profile">See code path</a></div>')
    f.write('<img id="1kpps_profile" src="data:image/svg+xml;base64,{}" />'.format(kpps_profile))
    f.write(biolatency_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(skblife_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(filetop_read_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(filetop_write_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(netsize_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(runqlat_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(cachestat_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(sormem_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(vfsstat_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(hardirqs_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(hardirqs_dist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(softirqs_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(softirqs_dist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(nettxlat_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(vfsstat_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(vfssize_fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(vfssize_fig_2.to_html(full_html=False, include_plotlyjs='cdn'))
    # f.write(vfssize_fig_3.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(sys_enter_brk_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(vfsstat_fig.to_html(full_html=False, include_plotlyjs='cdn'))